In [ ]:
import tensorflow as tf

# 导入Tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data

# 读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# 建立模型
## 构建输入层
### 定义标签数据占位符

In [ ]:
x = tf.placeholder(tf.float32, [None, 784], name="X")
y = tf.placeholder(tf.float32, [None, 10], name="Y")

## 构建隐藏层

In [ ]:
#隐藏层神经元数量
H1_NN = 256

W1 = tf.Variable(tf.random_normal([784, H1_NN]))
b1 = tf.Variable(tf.zeros([H1_NN]))

Y1 = tf.nn.relu(tf.matmul(x, W1) + b1)

## 构建输出层 

In [ ]:
W2 = tf.Variable(tf.random_normal([H1_NN, 10]))
b2 = tf.Variable(tf.zeros([10]))

forward = tf.matmul(Y1, W2) + b2
pred = tf.nn.softmax(forward)

# 训练模型
### 定义损失函数

In [ ]:
#交叉熵
#loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

### 设置训练参数

In [ ]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

### 选择优化器

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

### 定义准确率

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#记录训练开始时间
from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)  #读取批次数据
        sess.run(optimizer,feed_dict={x: xs, y: ys}) #执行批次训练
        
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率
    loss, acc = sess.run([loss_function, accuracy],feed_dict={x: mnist.validation.images,
                                                              y: mnist.validation.labels})
    if(epoch+1) % display_step ==0:
        print("Train Epoch:","%02d" % (epoch+1),
              "Loss=","{:.9f}".format(loss), " Accuracy=","{:.4f}".format(acc))
#运行总时间
duration = time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

# 评估模型

In [ ]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print("Test Accuracy:", accu_test)

# 进行预测

In [ ]:
#由于pred预测结果是one-hot编码格式，所以需要转换为0~9数字。
prediction_result=sess.run(tf.argmax(pred,1), feed_dict={x: mnist.test.images})

In [ ]:
#查看预测结果中的前10项
prediction_result[0:10]

# 定义可视化函数

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_images_labels_prediction(images,     #图像列表
                                  labels,     #标签列表
                                  prediction, #预测值列表
                                  index,      #从第index个开始显示
                                  num=10):    #缺省一次显示10幅
    fig = plt.gcf() #获取当前图表,Get Current Figure
    fig.set_size_inches(10,12) #1英寸等于2.54cm
    if num > 25:
        num = 25                #最多显示25个子图
    for i in range(0, num):
        ax = plt.subplot(5,5, i+1) #获取当前要处理的子图
        ax.imshow(np.reshape(images[index],(28, 28)),cmap='binary')
        
        title = "label=" + str(np.argmax(labels[index])) #构建该图上要显示的title
        
        if len(prediction)>0:
            title += ",predict=" + str(prediction[index])
            
        ax.set_title(title,fontsize=10) #显示图上的title信息
        ax.set_xticks([]); #不显示坐标轴
        ax.set_yticks([])
        index += 1
    plt.show()

In [ ]:
plot_images_labels_prediction(mnist.test.images,
                              mnist.test.labels,
                              prediction_result,10,25)

# 找出预测错误

In [ ]:
import numpy as np
compare_lists = prediction_result==np.argmax(mnist.test.labels,1) 
print(compare_lists)

In [ ]:
err_lists = [i for i in range(len(compare_lists)) if compare_lists[i]==False]
print(err_lists, len(err_lists))

# 定义一个输出错误分类的函数

In [ ]:
def print_predict_errs(labels,      #标签列表
                       prediction): #预测值列表
    count = 0
    compare_lists = (prediction==np.argmax(labels,1))
    err_lists = [i for i in range(len(compare_lists)) if compare_lists[i]==False]
    for x in err_lists:
        print("index="+str(x)+ 
              " 标签值=",np.argmax(labels[x]),
              "预测值=",prediction[x])
        count = count + 1
    print("总计:"+str(count))

In [ ]:
print_predict_errs(labels=mnist.test.labels,
                   prediction=prediction_result)